In [5]:
import pandas as pd
from pyinaturalist import get_observations
import time
import os
import shutil
from datetime import datetime
from requests.exceptions import HTTPError

def agregar_imagenes_a_csv(archivo_csv, fila_inicio=0, fila_fin=None, hacer_backup=True):
    """
    Añade la columna 'imagen' a un CSV con observaciones de iNaturalist
    
    Args:
        archivo_csv (str): Ruta al archivo CSV de entrada y salida
        fila_inicio (int): Número de fila desde donde comenzar el procesamiento
        fila_fin (int): Número de fila hasta donde procesar (None = hasta el final)
        hacer_backup (bool): Si es True, crea una copia de seguridad del archivo original
    """
    
    # Crear backup si se solicita
    if hacer_backup:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_file = f"{archivo_csv}.backup_{timestamp}"
        shutil.copy2(archivo_csv, backup_file)
        print(f"Backup creado en: {backup_file}")
    
    try:
        df = pd.read_csv(archivo_csv)
        print(f"CSV cargado: {len(df)} filas encontradas")
    except Exception as e:
        print(f"Error al cargar el CSV: {e}")
        return
    
    if 'imagen' not in df.columns:
        df['imagen'] = None
        print("Columna 'imagen' añadida al DataFrame")
    
    # Determinar la fila final si no se especifica
    if fila_fin is None:
        fila_fin = len(df) - 1
    else:
        # Asegurar que fila_fin no exceda el tamaño del DataFrame
        fila_fin = min(fila_fin, len(df) - 1)
    
    # Validar rangos
    if fila_inicio < 0 or fila_inicio > fila_fin or fila_fin >= len(df):
        print(f"Error: Rangos inválidos. Inicio: {fila_inicio}, Fin: {fila_fin}, Total filas: {len(df)}")
        return
    
    print(f"Comenzando procesamiento desde la fila {fila_inicio} hasta la fila {fila_fin}")
    print(f"Total de filas a procesar: {fila_fin - fila_inicio + 1}")
    
    for index, row in df.iterrows():
        # Saltar filas fuera del rango especificado
        if index < fila_inicio or index > fila_fin:
            continue
            
        if pd.notna(row.get('imagen')) and row['imagen'] != '' and not row['imagen'].startswith('Error 429'):
            print(f"Fila {index}: Ya tiene imagen, saltando...")
            continue
        
        observacion_id = row['id']
        print(f"Procesando fila {index}: ID {observacion_id}")
        
        while True:
            try:
                response = get_observations(id=observacion_id)
                
                if response['results']:
                    observation = response['results'][0]
                    
                    if observation['photos']:
                        photo_url = observation['photos'][0]['url'].replace('square', 'medium')
                        df.at[index, 'imagen'] = photo_url
                        print(f"  ✓ Imagen encontrada: {photo_url}")
                    else:
                        df.at[index, 'imagen'] = 'Sin imagen'
                        print(f"  ✗ Observación sin imágenes")
                else:
                    df.at[index, 'imagen'] = 'No encontrada'
                    print(f"  ✗ Observación no encontrada")
                
                # Si llegamos aquí, la solicitud fue exitosa, salimos del bucle while
                break
                    
            except HTTPError as e:
                if e.response.status_code == 429:
                    print(f"  ! Error 429: Demasiadas solicitudes. Esperando 30 segundos...")
                    time.sleep(30)
                    continue  # Continuar el bucle while para reintentar
                else:
                    df.at[index, 'imagen'] = f"Error HTTP {e.response.status_code}: {str(e)}"
                    print(f"  ! Error HTTP {e.response.status_code}: {e}")
                    break
                    
            except Exception as e:
                df.at[index, 'imagen'] = f"Error: {str(e)}"
                print(f"  ! Error al procesar: {e}")
                break
        
        df.to_csv(archivo_csv, index=False)
        print(f"  → Progreso guardado en {archivo_csv}")
        time.sleep(1)
    
    print(f"\nProceso completado! Archivo actualizado en: {archivo_csv}")
    return df

# Uso del código
if __name__ == "__main__":
    archivo_entrada = "ubicaciones.csv"
    
    fila_inicio = 9892
    fila_fin = 9900
    
    # Ejecutar el proceso
    df_resultado = agregar_imagenes_a_csv(
        archivo_entrada, 
        fila_inicio=fila_inicio, 
        fila_fin=fila_fin, 
        hacer_backup=True
    )
    
    # Mostrar resumen
    if df_resultado is not None:
        print("\nResumen final:")
        print(f"Total de filas procesadas: {len(df_resultado)}")
        print("Distribución de estados:")
        print(df_resultado['imagen'].value_counts())

SyntaxError: invalid syntax (627149713.py, line 115)